### SVM
支持向量机(Support Vector Machines, SVM): 是一种监督学习算法。
- 支持向量(Support Vector)就是离分隔超平面最近的那些点。
- 机(Machine)就是表示一种算法，而不是表示机器。
- SVM 能执行线性、非线性、回归、甚至是异常值检测的任务。它是机器学习最受欢迎的模型之一。
- SVM 特别适用于中小型复杂数据集的分类。
#### 原理
给定训练样本集$D=\{(x_1,y_1),(x_2,y_2),\dots,(x_m,y_m)\},y_i\in\{-1,+1\}$,其中$(x_i,y_i)$称为样本点。
当$y_i=1$时，$x_i$为正例
当$y_i=-1$时，$x_i$为反例

#### 超平面和最大间隔
![svm分类](src/svm_iris_classes.png)  
上左图中有三种可能的线性分类器决策边界：
- 虚线所代表的模型非常糟糕，甚至无法分类
- 其余两个在训练集上堪称完美，但是他们的边界离样本过近，加入新的样本后表现可能不好
上右图中实线代表SVM分类器的决策边界，不仅分离了两个类别，还尽可能原理最近的样本

#### 硬间隔分类
如果我们严格要求所有样本都不在最大间隔之内，并且位于正确的一边，那么这就是硬间隔。  
硬间隔有两个问题：
- 数据只有线性可分（一条直线可以将样本分开）才有效
- 对异常值非常敏感  
当有一个异常值的iris数据：
- 左图数据无法找出硬间隔
- 右图最终决策边界和无异常值时的决策边界也大不相同，无法有效的泛化  
![硬间隔的问题](src/svm_hard_margin.png)
#### 软间隔分类
缓解硬间隔带来的问题的一个方法是允许SVM在一些样本上出错。  
目标是尽可能在保持最大间隔宽度和限制间隔违例（即位于最大间隔之上，甚至在错误一边的样本）之间找到良好的平衡，这就是软间隔分类。  
在 Sklearn 中可以通过超参数 C 来控制间隔和违例，C越小，间隔越宽，违例越多，反之亦然。  
左图中使用了C=100,分类器的错误样本较少，间隔也较小
右图中使用了C=1,分类器的错误样本多了许多，间隔也变大了，右图的泛化能力比左图要更好。  
![软间隔](src/svm_soft_margin.png)

#### 目标函数
为了找到最大间隔的划分超平面，需要求解式如下:
$$
\underset{w,b}{min}\frac{1}{2}||w||^2\\
s.t.\quad y_i(w^Tx_i+b)\geq1,\quad i=1,2,\dots,m.
$$
这就是SVM的基本型
<!-- #### 求解目标函数
因为目标函数带有约束条件，需要利用拉格朗日乘子法求解。  
拉格朗日乘子法是一种寻找多元函数在一组约束下的极值得方法。  
引入拉格朗日乘子，可将d个变量与k个约束条件得最优化问题转化为具有d+k个变量得无约束优化问题求解。  
经过拉格朗日乘子，目标函数可以转换为：
$$L(w,b,\alpha)=\frac{1}{2}||w||^2+\sum_{i=1}^{m}\alpha_i(1-y_i(w^Tx_i+b))$$
其中，上式的后半部分 -->

#### 损失函数

![svm损失函数](src/svm_loss_func.png)

- 0/1损失
  - 正例点落在y=0超平面下，分类正确，无论距离超平面多远，误差为0
  - 正例点落在y=0超平面上，分类错误，无论距离超平面多远，误差为1
- hinge损失
  - 正例点落在y=1线上，距离超平面长度为1，那么$1-\xi=1,\xi=0,loss=0$
  - 正例点落距离超平面为0.5的位置，那么$1-\xi=0.5,\xi=0.5,loss=0.5$
  - 正例点落在y=0，距离超平面长度为0，那么$1-\xi=0,\xi=1,loss=1$
  - 正例点落在y=0上方，被误分类到负例中，距离算出来是负的，比如-0.5，那么$1-\xi=-0.5,\xi=-1.5,loss=1.5$
- logistic损失
  - 损失函数的公式为$ln(1+e^{-yi})$

#### 核函数

![核函数](src/kernel_function.png)

假设$X$是输入空间，$H$是特征空间，存在一个映射$\phi$使得$X$中的点$x$能够计算得到$H$空间中的点$h$，对于所有的$X$中的点都成立：
$$h = \phi(x)$$
若$x,z$是$X$空间中的点，函数$k(x,z)$满足下述条件，那么都成立，则称$k$为核函数，而$\phi$为映射函数
$$k(x,z)=\phi(x)\cdot\phi(z)$$


前面的讨论，我们假设训练样本是线性可分的，但在现实中，许多数据并不能线性可分。    
对于这样的问题，可将样本从原始空间映射到一个高维特征空间，使得样本在高维特征空间中线性可分。  
在特征空间中划分超平面所用的模型为：
$$f(x)=w^T\phi(x)+b$$
当中的$\phi(x)$称为***核函数***，是某个确定的特征空间转换函数，它的作用是将x映射到更高的维度。
其实也就是要求一组参数$(w,b)$，使其构建的超平面函数能够最优地分离两个集合。

核函数的作用就是一个从低维空间到高维空间的映射，而这个映射可以把低维空间中线性不可分的两类点变成线性可分。  
##### 常见核函数
- 线性核
  - $k(x_i,x_j)=x_i^Tx_j$
- 多项式核
  - $k(x_i,x_j)=(x_i^Tx_j)^d$
  - $d\geq1$为多项式的次数
  - $d=1$时退化为线性核
- 高斯核 - RBF - 最常用
  - $k(x_i,x_j)=exp(-\frac{||x_i-x_j||^2}{2\sigma^2})$
  - $\sigma>0$为高斯核的带宽(width)
- 拉普拉斯核
  - $k(x_i,x_j)=exp(-\frac{||x_i-x_j||}{\sigma})$
  - $\sigma>0$
- sigmoid核
  - $k(x_i,x_j)=tanh(-{\beta}x_i^Tx_j+\theta)$
  - $tanh$为双曲线正切函数，${\beta}>0,\theta<0$

- 线性和多项式核
  - 首先在属性空间中找到一些点，把这些点当作基点（base）,核函数的作用是找与该点距离和角度满足某种关系的样本点
  - 当样本点与该点的夹角近乎垂直时，两个样本的欧式长度必须非常长才能保证满足线性核函数大于0；而当样本点与base点的方向相同时，长度就不必很长；而当方向相反时，核函数值就是负的，被判为反类。即，它在空间上划分出一个梭形，按照梭形来进行正反类划分。
- RBF 高斯核
  - 高斯核函数就是在属性空间中找到一些点，这些点能够是也能够不是样本点，把这些点当作base，以这些base为圆心向外扩展，扩展半径即为带宽，便可划分数据。
  - 换句话说，在属性空间中找到一些超圆，用这些超圆来断定正反类。
  - 画一个圆，圆内一个类，圆外一个类
- Sigmoid核：
  - 一样地是定义一些base。
  - 核函数就是将线性核函数通过一个tanh函数进行处理，把值域限制在了-1到1上。
- 总之，都是在定义距离，大于该距离，判为正，小于该距离，判为负。至于选择哪种核函数，要根据具体的样本分布状况来肯定。
- 指导规则
  - 如果 Feature 的数量很大，跟样本数量差不多，这时候选用 LR 或者是 Linear Kernel 的 SVM
  - 如果 Feature 的数量比较小，样本数量一般，不算大也不算小，选用 SVM+Gaussian Kernel
  - 如果 Feature 的数量比较小，而样本数量很多，需要手工添加一些 feature 变成第一种情况
  - 多项式核一般很少使用，效率也不高，结果也不优于RBF
  - Linear Kernel参数少，速度快
  - RBF参数多，分类结果非常依赖参数设置，需要GridSearch和cross-validate，比较耗时
  - 应用最广的是RBF,无论样本多少，维度高低，RBF都很适用

#### SVM 回归
SVM回归(support vector regression)是让尽可能多的实例位于预测线上，同时限制间隔违例（也就是不在线上的实例），线距的宽度有$\epsilon$决定

![SVR](src/svr.png)

#### API
- SVM的方法可以用于分类（二分类/多分类），也可以用于回归值和异常值检测
- SVM有良好的鲁棒性，对未知数据有很强的泛化能力，特别是在数据量较少的情况下，相较于传统机器学习算法有更好的性能
##### 通用流程
- 对样本进行无量纲化
- 应用核函数对样本进行映射（最常用为RBF和线性核，在样本线性可分时，线性核比RBF效果好）
- 用GridSearch和cross-validate对超参数进行搜索
- 用最优参数训练模型
- 模型评估
##### 主要方法
~~~python
sklearn.svm.SVC # C-Support Vector Classification.
sklearn.svm.NuSVC # Nu-Support Vector Classification.
sklearn.svm.LinearSVC # Linear Support Vector Classification.
~~~
- SVC 和 NuSVC 的方法基本一致，区别是损失函数的度量方式不同
  NuSVC 的 nu 参数是 SVC 中的 C 参数
- LinearSVC 是线性核的支持向量分类，没有 kernel 参数

- SVC
  - C: float参数 默认值为1.0
    - 错误项的惩罚系数。
    - C越大，即对分错样本的惩罚程度越大，因此在训练样本中准确率越高，但是泛化能力降低，也就是对测试数据的分类准确率降低，容易过拟合。
    - C越小，即容许训练样本中有一些误分类错误样本，泛化能力强，容易欠拟合。
    - 对于训练样本带有噪声的情况，一般采用后者，把训练样本集中错误分类的样本作为噪声。
  - kernel: str参数 默认为‘rbf’
    - 算法中采用的核函数类型，可选参数有：
    - ‘linear’:线性核函数
    - ‘poly’：多项式核函数
    - ‘rbf’：径像核函数/高斯核
    - ‘sigmod’:sigmod核函数
    - ‘precomputed’:核矩阵
      - precomputed表示自己提前计算好核函数矩阵，这时候算法内部就不再用核函数去计算核矩阵，而是直接用你给的核矩阵。
  - degree: int型参数 默认为3
    - 这个参数只对多项式核函数有用，是指多项式核函数的阶数n
    - 如果给的核函数参数是其他核函数，则会自动忽略该参数。
  - coef0: float参数 默认为0.0
    - y=kx+b中的b值
    - 核函数中的独立项，只有对‘poly’和‘sigmod’核函数有用，是指其中的参数c
- NuSVC
  - nu: 训练误差部分的上限和支持向量部分的下限，取值(0,1)，默认0.5
- LinearSVC
  - penalty : string, ‘l1’ or ‘l2’ (default=’l2’)
    - 指定惩罚中使用的规范。 'l2'惩罚是SVC中使用的标准。 'l1'导致稀疏的coef_向量。
  - loss : string, ‘hinge’ or ‘squared_hinge’ (default=’squared_hinge’)
    - 指定损失函数。 “hinge”是标准的SVM损失，类似于L1损失（例如由SVC类使用），而“squared_hinge”是hinge损失的平方。类似于L2。
  - dual : bool, (default=True)
    - 是否转换为对偶问题求解
  - C : float, optional (default=1.0)
    - 错误项的惩罚参数，类似于线性回归的正则化系数

In [1]:
from sklearn.svm import SVC

X = [[1,1],[2,3]]
y=[0,1]
clf = SVC()
clf.fit(X, y)


SVC()

In [4]:
clf.predict([[5,5]])

array([1])